In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.model_selection import train_test_split

In [2]:
def vgg_net(Images=None,reuse=False,sess=None):
    #Discriminator
    Parameters=[]
    with tf.variable_scope(tf.contrib.framework.get_name_scope(), reuse=reuse):
        #Placeholder
        if Images==None:
            Images=tf.placeholder(tf.float32,shape=(None,224,224,3),name='Dinput')
        vggmean=tf.constant([123.68,116.779,103.939], dtype=tf.float32, shape=[1,1,1,3])
        Images=(Images-vggmean)
        #Conv1
        W1=tf.get_variable("DW1",[3,3,3,64],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b1=tf.get_variable("Db1",[64],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv1=tf.nn.conv2d(Images,W1,[1,1,1,1],padding='SAME',name='Dconv1')
        z1=tf.nn.bias_add(conv1,b1,name='Dz1')
        a1=tf.nn.relu(z1,name='Da1')
        Parameters+=[W1,b1]
        #Conv2
        W2=tf.get_variable("DW2",[3,3,64,64],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b2=tf.get_variable("Db2",[64],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv2=tf.nn.conv2d(a1,W2,[1,1,1,1],padding='SAME',name='Dconv2')
        z2=tf.nn.bias_add(conv2,b2,name='Dz2')
        a2=tf.nn.relu(z2,name='Da2')
        Parameters+=[W2,b2]
        #Pool1
        pool1=tf.nn.max_pool(a2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool1')
        #Conv3
        W3=tf.get_variable("DW3",[3,3,64,128],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b3=tf.get_variable("Db3",[128],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv3=tf.nn.conv2d(pool1,W3,[1,1,1,1],padding='SAME',name='Dconv3')
        z3=tf.nn.bias_add(conv3,b3,name='Dz3')
        a3=tf.nn.relu(z3,name='Da3')
        Parameters+=[W3,b3]
        #Conv4
        W4=tf.get_variable("DW4",[3,3,128,128],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b4=tf.get_variable("Db4",[128],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv4=tf.nn.conv2d(a3,W4,[1,1,1,1],padding='SAME',name='Dconv4')
        z4=tf.nn.bias_add(conv4,b4,name='Dz4')
        a4=tf.nn.relu(z4,name='Da4')
        Parameters+=[W4,b4]
        #Pool2
        pool2=tf.nn.max_pool(a4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool2')
        #Conv5
        W5=tf.get_variable("DW5",[3,3,128,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b5=tf.get_variable("Db5",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv5=tf.nn.conv2d(pool2,W5,[1,1,1,1],padding='SAME',name='Dconv5')
        z5=tf.nn.bias_add(conv5,b5,name='Dz5')
        a5=tf.nn.relu(z5,name='Da5')
        Parameters+=[W5,b5]
        #Conv6
        W6=tf.get_variable("DW6",[3,3,256,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b6=tf.get_variable("Db6",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv6=tf.nn.conv2d(a5,W6,[1,1,1,1],padding='SAME',name='Dconv6')
        z6=tf.nn.bias_add(conv6,b6,name='Dz6')
        a6=tf.nn.relu(z6,name='Da6')
        Parameters+=[W6,b6]
        #Conv7
        W7=tf.get_variable("DW7",[3,3,256,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b7=tf.get_variable("Db7",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv7=tf.nn.conv2d(a6,W7,[1,1,1,1],padding='SAME',name='Dconv7')
        z7=tf.nn.bias_add(conv7,b7,name='Dz7')
        a7=tf.nn.relu(z7,name='Da7')
        Parameters+=[W7,b7]
        #Pool3
        pool3=tf.nn.max_pool(a7,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name='Dpool3')
        #Conv8
        W8=tf.get_variable("DW8",[3,3,256,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b8=tf.get_variable("Db8",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv8=tf.nn.conv2d(pool3,W8,[1,1,1,1],padding='SAME',name='Dconv8')
        z8=tf.nn.bias_add(conv8,b8,name='Dz8')
        a8=tf.nn.relu(z8,name='Da8')
        Parameters+=[W8,b8]
        #Conv9
        W9=tf.get_variable("DW9",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b9=tf.get_variable("Db9",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv9=tf.nn.conv2d(a8,W9,[1,1,1,1],padding='SAME',name='Dconv9')
        z9=tf.nn.bias_add(conv9,b9,name='Dz9')
        a9=tf.nn.relu(z9,name='Da9')
        Parameters+=[W9,b9]
        #Conv10
        W10=tf.get_variable("DW10",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b10=tf.get_variable("Db10",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv10=tf.nn.conv2d(a9,W10,[1,1,1,1],padding='SAME',name='Dconv10')
        z10=tf.nn.bias_add(conv10,b10,name='Dz10')
        a10=tf.nn.relu(z10,name='Da10')
        Parameters+=[W10,b10]
        #Pool4
        pool4=tf.nn.max_pool(a10,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool4')
        #Conv11
        W11=tf.get_variable("DW11",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b11=tf.get_variable("Db11",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv11=tf.nn.conv2d(pool4,W11,[1,1,1,1],padding='SAME',name='Dconv11')
        z11=tf.nn.bias_add(conv11,b11,name='Dz11')
        a11=tf.nn.relu(z11,name='Da11')
        Parameters+=[W11,b11]
        #Conv12
        W12=tf.get_variable("DW12",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b12=tf.get_variable("Db12",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv12=tf.nn.conv2d(a11,W12,[1,1,1,1],padding='SAME',name='Dconv12')
        z12=tf.nn.bias_add(conv12,b12,name='Dz12')
        a12=tf.nn.relu(z12,name='Da12')
        Parameters+=[W12,b12]
        #Conv13
        W13=tf.get_variable("DW13",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b13=tf.get_variable("Db13",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv13=tf.nn.conv2d(a12,W13,[1,1,1,1],padding='SAME',name='Dconv13')
        z13=tf.nn.bias_add(conv13,b13,name='Dz13')
        a13=tf.nn.relu(z13,name='Da13')
        Parameters+=[W13,b13]
        #Pool5
        pool5=tf.nn.max_pool(a13,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool5')
        
        #fc1
        shape = int(np.prod(pool5.get_shape()[1:]))
        print(shape)
        fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights',trainable=True)
        fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                                   name='biases',trainable=True)
        pool5a_flat = tf.reshape(pool5, [-1, shape])
        za = tf.nn.bias_add(tf.matmul(pool5a_flat, fc1w), fc1b)
        aa = tf.nn.relu(za,name='Fc1_a')
        Parameters += [fc1w, fc1b]
        
        #fc2
        shape = int(np.prod(aa.get_shape()[1:]))
        fc2w = tf.Variable(tf.truncated_normal([shape, 1000],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights',trainable=True)
        fc2b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
                                   name='biases',trainable=True)
        pool5b_flat = tf.reshape(aa, [-1, shape])
        zb = tf.nn.bias_add(tf.matmul(pool5b_flat, fc2w), fc2b)
        ab = tf.nn.relu(zb,name='Fc2_b')
        Parameters += [fc2w, fc2b]
        
        #fc3
        shape = int(np.prod(ab.get_shape()[1:]))
        fc3w = tf.Variable(tf.truncated_normal([shape, 128],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights',trainable=True)
        fc3b = tf.Variable(tf.constant(1.0, shape=[128], dtype=tf.float32),
                                   name='biases',trainable=True)
        pool5c_flat = tf.reshape(ab, [-1, shape])
        zc = tf.nn.bias_add(tf.matmul(pool5c_flat, fc3w), fc3b)
        ac = tf.nn.relu(zc,name='Fc3_c')
        Parameters += [fc3w, fc3b]
        
        
        return ac

In [3]:
#Placeholder - Anchors
sess = tf.Session()
AnchorImages=tf.placeholder(tf.float32,shape=(None,224,224,3),name='Anchorinput')
#Images = pickle.load( open( "v1TrainBatch1.pkl", "rb" ) )
#Placeholder - Positives
PositiveImages=tf.placeholder(tf.float32,shape=(None,224,224,3),name='Positiveinput')

#Placeholder - Negatives
NegativeImages=tf.placeholder(tf.float32,shape=(None,224,224,3),name='Negativeinput')

In [4]:
Anchor = vgg_net(Images=AnchorImages,sess=sess)
Positive = vgg_net(Images=PositiveImages, reuse=True)
Negative = vgg_net(Images=NegativeImages, reuse=True)

25088
25088
25088


In [ ]:
print(Anchor,Positive, Negative)

Tensor("Fc3_c:0", shape=(?, 128), dtype=float32) Tensor("Fc3_c_1:0", shape=(?, 128), dtype=float32) Tensor("Fc3_c_2:0", shape=(?, 128), dtype=float32)


In [ ]:
writer = tf.summary.FileWriter("Graph", sess.graph)
writer.close()

In [5]:
def EuclideanDistance(a,b):
    l2_norm = tf.reduce_mean(tf.norm(a-b,axis=1, ord='euclidean'))
    return l2_norm

In [7]:
alpha = 0.7
loss = tf.maximum((EuclideanDistance(Anchor,Positive) - EuclideanDistance(Anchor,Negative)+ alpha), 0, name="loss")

In [8]:
loss

<tf.Tensor 'loss:0' shape=() dtype=float32>

In [9]:
def train(optimizer, sess, loss,numberofepochs=1, batchsize=8):
    str1='v2TrainBatch_224_224_'
    for _ in range(numberofepochs):
        for i in range(1,101):
            Dict=pickle.load(open(str1+str(i)+".pkl",'rb'))
            A1=Dict['Anchor']
            P=Dict['Positives']
            N=Dict['Negatives']
            for j in range(0,A1.shape[0],batchsize):
                batchA = A1[j:j+batchsize,:,:,:]
                batchP = P[j:j+batchsize,:,:,:]
                batchN = N[j:j+batchsize,:,:,:]
                temp, c = sess.run([optimizer, loss], feed_dict = {AnchorImages:batchA, PositiveImages:batchP, NegativeImages:batchN})
                print(j, c)

In [10]:
optimizer=tf.train.AdamOptimizer(0.001).minimize(loss)

In [14]:
sess.run(tf.global_variables_initializer())

ResourceExhaustedError: OOM when allocating tensor of shape [25088,4096] and type float
	 [[Node: weights/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [25088,4096] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'weights/Adam/Initializer/zeros', defined at:
  File "c:\users\sai rathan\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\sai rathan\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\sai rathan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\sai rathan\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\sai rathan\appdata\local\programs\python\python

In [ ]:
train(optimizer,sess,loss)